# 2.0 Data
This notebook does extracts the variables of interest, splits the data into training, validation and test sets, then saves the data for downstream analysis and modeling.  


In [1]:
# libraries
import os
import sys
import inspect
sys.path.append("../src")

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from shared import directories
from shared import filenames
from shared import variables

### Obtain raw data

In [2]:
df = pd.read_csv(os.path.join(directories.RAW_DATA_DIR,
                              filenames.RAW_FILENAME),
                 encoding="Latin-1", low_memory=False)

### Extract variables of interest

In [3]:
omit = ['field', 'has_null']
df = df.drop(columns=omit)

### Create categorical variables for binary numeric variables 

In [4]:
df['samerace_c'] = np.where(df['samerace']==1, 'yes', 'no')
df['decision_c'] = np.where(df['decision']==1, 'yes', 'no')
df['decision_o_c'] = np.where(df['decision_o']==1, 'yes', 'no')
df['match_c'] = np.where(df['match']==1, 'yes', 'no')
df.head()


,wave,gender,age,age_o,d_age,d_d_age,race,race_o,samerace,importance_same_race,...,d_like,d_guess_prob_liked,met,decision,decision_o,match,samerace_c,decision_c,decision_o_c,match_c
0,1,female,21.0,27.0,6,[4-6],asian/pacific islander/asian-american,european/caucasian-american,0,2.0,...,[6-8],[5-6],0.0,1,0,0,no,yes,no,no
1,1,female,21.0,22.0,1,[0-1],asian/pacific islander/asian-american,european/caucasian-american,0,2.0,...,[6-8],[5-6],1.0,1,0,0,no,yes,no,no
2,1,female,21.0,22.0,1,[0-1],asian/pacific islander/asian-american,asian/pacific islander/asian-american,1,2.0,...,[6-8],[0-4],1.0,1,1,1,yes,yes,yes,yes
3,1,female,21.0,23.0,2,[2-3],asian/pacific islander/asian-american,european/caucasian-american,0,2.0,...,[6-8],[5-6],0.0,1,1,1,no,yes,yes,yes
4,1,female,21.0,24.0,3,[2-3],asian/pacific islander/asian-american,latino/hispanic american,0,2.0,...,[6-8],[5-6],0.0,1,1,1,no,yes,yes,yes


### Correct / Normalize Feature Names

In [5]:
df.rename({'sinsere_o': 'sincere_o',
           'intellicence_important': 'intelligence_important',
           'ambtition_important': 'ambitious_important',
           'ambition_partner': 'ambitious_partner',
           'ambition' : 'ambitious',
           'ambitous_o' : 'ambitious_o'},
          inplace=True,
          axis='columns')

### Recode Race Levels
Recode race labels for easy plotting and correct misspelled column names

In [6]:
df['race'] = df['race'].replace({
    'asian/pacific islander/asian-american': 'asian',
    'european/caucasian-american': 'caucasian',
    'black/african american': 'black',
    'latino/hispanic american': 'latino',
    'other': 'other'})
df['race_o'] = df['race_o'].replace({
    'asian/pacific islander/asian-american': 'asian',
    'european/caucasian-american': 'caucasian',
    'black/african american': 'black',
    'latino/hispanic american': 'latino',
    'other': 'other'})

## Features
### Participants per Wave
Here we capture the number of males and females per wave.

In [7]:
df2 = df[df['gender'] == 'male'][['gender', 'wave', 'match']].groupby(['wave', 'gender']).agg(['count']).reset_index()
df3 = df[df['gender'] == 'female'][['gender', 'wave', 'match']].groupby(['wave', 'gender']).agg(['count']).reset_index()
df2.columns =['wave', 'gender', 'wave_males']
df3.columns =['wave2', 'gender2', 'wave_females']
df4 = pd.concat([df2, df3], axis=1)
df4 = df4[['wave', 'wave_males', 'wave_females']]
df = pd.merge(df, df4, on='wave')

### Difference in Perceptions
The differences between self and partner perceptions along the six characteristics being studies. Specifically, interested in
    1. the differences between the subjects self-perception and their partner's assessment of them across the six
    characteristics being studied, and    
    2. the differences between the subjects self-perceptions and their perceptions of their partner

In [8]:
# Compute subject perception of relative differences
df['rd_attractive'] = (df['attractive_partner'] - df['attractive']) / df['attractive']
df['rd_sincere'] = (df['sincere_partner'] - df['sincere']) / df['sincere']
df['rd_intelligence'] = (df['intelligence_partner'] - df['intelligence']) / df['intelligence']
df['rd_funny'] = (df['funny_partner'] - df['funny']) / df['funny']
df['rd_ambitious'] = (df['ambitious_partner'] - df['ambitious']) / df['ambitious']

# Compute relative difference in subject and partner impressions
df['rd_attractive_o'] = (df['attractive_o'] - df['attractive']) / df['attractive']
df['rd_sincere_o'] = (df['sincere_o'] - df['sincere']) / df['sincere']
df['rd_intelligence_o'] = (df['intelligence_o'] - df['intelligence']) / df['intelligence']
df['rd_funny_o'] = (df['funny_o'] - df['funny']) / df['funny']
df['rd_ambitious_o'] = (df['ambitious_o'] - df['ambitious']) / df['ambitious']   

### Split into training, validation and test sets. 
First split data set into a training/test sets 80/20. Then split training set into training/validation sets 80/20

In [9]:
train, test = train_test_split(df, test_size=0.2, random_state=5)
train, validation = train_test_split(train, test_size=0.2, random_state=5)

### Save data 
Save training, validation and test sets in an interim directory for exploratory data analysis

In [10]:
def write(df, directory, filename):
    if isinstance(df, pd.DataFrame):
        if isinstance(filename, str):
            if not os.path.isdir(directory):
                os.mkdir(directory)
            df.to_csv(os.path.join(directory, filename),
                      index=False, index_label=False)
            return(True)
        else:
            return(False)
    else:
        return(False)

In [11]:
write(train, directories.INTERIM_DATA_DIR, filenames.TRAIN_FILENAME)
write(validation, directories.INTERIM_DATA_DIR, filenames.VALIDATION_FILENAME)
write(test, directories.INTERIM_DATA_DIR, filenames.TEST_FILENAME)

True